In [1]:
import datetime
import sys
import pandas as pd
import numpy as np
import sklearn

# 读取数据
data  = pd.read_csv('dataset_temp/okxorderbook20240421_1_cleaned.csv')

def unique_second(data):
    # 每一秒只保留一个数据
    data['datetime'] = pd.to_datetime(data['server_time'], unit='ms')
    # 设置server_time为索引
    data.set_index('datetime', inplace=True)
    # 按秒对server_time进行分组，并选择每个组的第一个记录
    df_unique_second = data.groupby(data.index.floor('S')).first()
    # 如果需要，可以将结果转换回DataFrame，并重置索引
    df_unique_second = df_unique_second.reset_index()
    return df_unique_second


data = unique_second(data)

# 当前价格，就是卖一价
data['price'] = data['ask_1_price']



/var/folders/14/djzvbv312951ytxybb9bdcb00000gn/T/ipykernel_56435/2190754619.py:16: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_unique_second = data.groupby(data.index.floor('S')).first()


In [2]:
def get_price_rise_radio(data):
    # 从5秒 到 60 秒，步长为5秒
    for i in range(5,60,5 ):
        data['rise_radio_' + str(i)] = data['price'].pct_change(i)
    return data

data = get_price_rise_radio(data)
data.to_csv('dataset_temp/okxorderbook20240421_1_cleaned_rise_radio.csv', index=False)